<a href="https://colab.research.google.com/github/Skyline-9/U2-Background-Removal/blob/main/U2_Background_Removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Learning Based Background Removal Using U<sup>2</sup> - Net**
By Richard Luo

*Since: June 25, 2021*

Link to research article: https://arxiv.org/pdf/2005.09007.pdf


### **Verifying GPU**
Make sure GPU acceleration is enabled, preferably the T4 over the P1000.

In [8]:
!nvidia-smi

Tue Jun 29 01:00:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Mounting Google Drive**

In [9]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Testing this first out with other code**



In [3]:
%cd '/content/gdrive/MyDrive'
!git clone https://github.com/Nkap23/background_removal_DL.git

/content/gdrive/MyDrive
Cloning into 'background_removal_DL'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 86 (delta 26), reused 54 (delta 11), pack-reused 0
Unpacking objects: 100% (86/86), done.


Installing dependencies

In [10]:
from cv2 import cv2 
from PIL import Image

Creating temporary folder

In [5]:
%cd '/content/gdrive/MyDrive/background_removal_DL/test_data/images'
!mkdir u2net_results

/content/gdrive/MyDrive/background_removal_DL/test_data/images


Run Salient Object Detection

In [12]:
%cd '/content/gdrive/MyDrive/background_removal_DL'
!python u2net_image.py

/content/gdrive/MyDrive/background_removal_DL
['/content/gdrive/MyDrive/background_removal_DL/test_data/images/input/input.jpg']
...load U2NET---173.6 MB
inferencing: input.jpg
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False sin

Bitwise Subtraction

In [13]:
#u2netresult
u2netresult=cv2.imread('/content/gdrive/MyDrive/background_removal_DL/test_data/images/u2net_results/input.png')
#orginalimage (CHANGE FILE EXTENSION HERE - BY DEFAULT: .jpg)
original=cv2.imread('/content/gdrive/MyDrive/background_removal_DL/test_data/images/input/input.jpg')
#subimage
subimage=cv2.subtract(u2netresult,original)
cv2.imwrite('/content/gdrive/MyDrive/background_removal_DL/test_data/images/output/output.png',subimage)

True

Extracting Original

In [14]:
#subimage
subimage=Image.open('/content/gdrive/MyDrive/background_removal_DL/test_data/images/output/output.png')
#originalimage
original=Image.open('/content/gdrive/MyDrive/background_removal_DL/test_data/images/input/input.jpg')

subimage=subimage.convert("RGBA")
original=original.convert("RGBA")

subdata=subimage.getdata()
ogdata=original.getdata()

newdata=[]
for i in range(subdata.size[0]*subdata.size[1]):
  if subdata[i][0]==0 and subdata[i][1]==0 and subdata[i][2]==0:
    newdata.append((255,255,255,0))
  else:
    newdata.append(ogdata[i])
subimage.putdata(newdata)
subimage.save('/content/gdrive/MyDrive/background_removal_DL/test_data/images/output/output.png',"PNG")
